In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/stdcode2024/Session07 - 유형별로 실습하는 머신러닝

/content/drive/MyDrive/Colab Notebooks/stdcode2024/Session07 - 유형별로 실습하는 머신러닝


In [3]:
import pandas as pd

In [4]:
### Step 1: 데이터를 바꾸더라도 혼자 할 수 있는 힘
### Step 2: 분석 시 카테고리에 대한 이해
### Step 3: 모델을 바꿔서 적용

In [5]:
secData = pd.read_csv("../dataset/kopo_decision_tree_all_new.csv")

### 1. 데이터 타입 통합 및 특성 숫자 컬럼 추가

In [6]:
secData.dtypes

REGIONID         object
PRODUCTGROUP     object
PRODUCT          object
ITEM             object
YEARWEEK          int64
YEAR              int64
WEEK              int64
QTY               int64
HOLIDAY          object
HCLUS             int64
PROMOTION        object
PRO_PERCENT     float64
dtype: object

In [7]:
## 데이터 타입 통합 skip

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
ynLabel = LabelEncoder()

In [10]:
secData["LE_HOLIDAY"] = ynLabel.fit_transform( secData.HOLIDAY ) # 숫자로 바꿀 컬럼 선정
secData["LE_PROMOTION"] = ynLabel.fit_transform( secData.PROMOTION )

### 2. 특성 선정 / 데이터 분리

In [11]:
corrDf = secData.corr(numeric_only=True)
corrDf # QTY에 대해 상관관계 0.5 이상인 애가 하나도 없음

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION
YEARWEEK,1.000000,0.984528,0.005567,-0.021432,0.016040,0.161164,0.001566,0.118040
YEAR,0.984528,1.000000,-0.169746,-0.039309,0.068345,0.122548,-0.041092,0.097886
WEEK,0.005567,-0.169746,1.000000,0.103788,-0.299818,0.207041,0.243307,0.105250
QTY,-0.021432,-0.039309,0.103788,1.000000,-0.184016,0.237682,0.157853,0.095984
HCLUS,0.016040,0.068345,-0.299818,-0.184016,1.000000,-0.289396,-0.977417,-0.143294
PRO_PERCENT,0.161164,0.122548,0.207041,0.237682,-0.289396,1.000000,0.254015,0.812344
LE_HOLIDAY,0.001566,-0.041092,0.243307,0.157853,-0.977417,0.254015,1.000000,0.136497
LE_PROMOTION,0.118040,0.097886,0.105250,0.095984,-0.143294,0.812344,0.136497,1.000000


In [12]:
stdCorr = 0.5

In [13]:
secData # 데이터를 한 번 봐야 함! 이 데이터로 내가 뭘 할 수 있지?
# ITEM이 서로 다 다르네? 머신러닝을 ITEM 단위로 돌려보자! (ITEM을 최소단위로! => ITEM의 특징을 제대로 살려서 돌릴 수 있다!)
# Group by (같은 그룹 키 별로 평균/sum)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION
0,A01,PG01,P01,ITEM001,201538,2015,38,1,N,4,N,0.0,0,0
1,A01,PG01,P01,ITEM001,201548,2015,48,1,Y,0,N,0.0,1,0
2,A01,PG01,P01,ITEM001,201549,2015,49,2,Y,0,N,0.0,1,0
3,A01,PG01,P01,ITEM002,201526,2015,26,1,Y,1,N,0.0,1,0
4,A01,PG01,P01,ITEM002,201532,2015,32,1,N,4,N,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32410,A01,PG05,P16,ITEM491,201503,2015,3,1,N,4,N,0.0,0,0
32411,A01,PG05,P16,ITEM491,201504,2015,4,1,Y,2,N,0.0,1,0
32412,A01,PG05,P16,ITEM491,201505,2015,5,2,N,4,N,0.0,0,0
32413,A01,PG05,P16,ITEM491,201506,2015,6,1,N,4,N,0.0,0,0


In [14]:
groupKey = ["REGIONID", "ITEM"] # 각 지역별 아이템의 특성을 살리기 위해 그룹키에 REGIONID 추가

In [15]:
secData.ITEM.drop_duplicates() # 아이템 종류 개수 확인 => 491개! 너무 많다

0        ITEM001
3        ITEM002
5        ITEM003
6        ITEM004
13       ITEM005
          ...   
32164    ITEM487
32195    ITEM488
32224    ITEM489
32318    ITEM490
32410    ITEM491
Name: ITEM, Length: 491, dtype: object

In [16]:
### 머신러닝의 한계: 그룹별로 건수가 많지 않으면 아예 머신러닝 불가능!

### 머신러닝이 할 수 있는 건 어떻게 나누는지

In [17]:
groupData = secData.groupby( by=groupKey )[["QTY"]].agg("count").reset_index()

In [18]:
groupData = groupData.rename(columns={"QTY": "QTY_COUNT"})
groupData # 첫 번째 행: A01 지역에 ITEM001 판매량 건수 3개밖에 없음 => 머신러닝 못 돌림!

,REGIONID,ITEM,QTY_COUNT
0,A01,ITEM001,3
1,A01,ITEM002,2
2,A01,ITEM003,1
3,A01,ITEM004,7
4,A01,ITEM005,8
...,...,...,...
486,A01,ITEM487,31
487,A01,ITEM488,29
488,A01,ITEM489,94
489,A01,ITEM490,92


In [19]:
mergedData = pd.merge(left = secData,
         right = groupData,
         on = groupKey,
         how = "left")

In [20]:
# 1년 기준 52주 데이터가 없으면 머신러닝 안 함! 104주로 해서 2년치 데이터 써도 됨~
# maxKnob = 52
maxKnob = 104

In [21]:
nomlData = mergedData.loc[ mergedData.QTY_COUNT < maxKnob ] # 분석 못하는 데이터
nomlData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION,QTY_COUNT
0,A01,PG01,P01,ITEM001,201538,2015,38,1,N,4,N,0.0,0,0,3
1,A01,PG01,P01,ITEM001,201548,2015,48,1,Y,0,N,0.0,1,0,3
2,A01,PG01,P01,ITEM001,201549,2015,49,2,Y,0,N,0.0,1,0,3
3,A01,PG01,P01,ITEM002,201526,2015,26,1,Y,1,N,0.0,1,0,2
4,A01,PG01,P01,ITEM002,201532,2015,32,1,N,4,N,0.0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32410,A01,PG05,P16,ITEM491,201503,2015,3,1,N,4,N,0.0,0,0,5
32411,A01,PG05,P16,ITEM491,201504,2015,4,1,Y,2,N,0.0,1,0,5
32412,A01,PG05,P16,ITEM491,201505,2015,5,2,N,4,N,0.0,0,0,5
32413,A01,PG05,P16,ITEM491,201506,2015,6,1,N,4,N,0.0,0,0,5


In [22]:
mlData = mergedData.loc[ mergedData.QTY_COUNT >= maxKnob ] # 분석 가능한 데이터 (구분해서 알려줄 수 있어야 한다!)
mlData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION,QTY_COUNT
419,A01,PG01,P01,ITEM013,201501,2015,1,6,Y,1,N,0.000000,1,0,130
420,A01,PG01,P01,ITEM013,201502,2015,2,4,N,4,N,0.000000,0,0,130
421,A01,PG01,P01,ITEM013,201503,2015,3,7,N,4,N,0.000000,0,0,130
422,A01,PG01,P01,ITEM013,201504,2015,4,17,Y,2,N,0.000000,1,0,130
423,A01,PG01,P01,ITEM013,201505,2015,5,7,N,4,N,0.000000,0,0,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31458,A01,PG05,P16,ITEM465,201701,2017,1,1090,Y,1,Y,0.308385,1,1,110
31459,A01,PG05,P16,ITEM465,201702,2017,2,884,N,4,Y,0.308385,0,1,110
31460,A01,PG05,P16,ITEM465,201703,2017,3,347,Y,2,Y,0.308385,1,1,110
31461,A01,PG05,P16,ITEM465,201712,2017,12,41,N,4,N,0.000000,0,0,110


In [23]:
mlData["IDX_KEY"] = mlData.REGIONID + "_" + mlData.ITEM
mlData

<ipython-input-23-6e604061223e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlData["IDX_KEY"] = mlData.REGIONID + "_" + mlData.ITEM


,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION,QTY_COUNT,IDX_KEY
419,A01,PG01,P01,ITEM013,201501,2015,1,6,Y,1,N,0.000000,1,0,130,A01_ITEM013
420,A01,PG01,P01,ITEM013,201502,2015,2,4,N,4,N,0.000000,0,0,130,A01_ITEM013
421,A01,PG01,P01,ITEM013,201503,2015,3,7,N,4,N,0.000000,0,0,130,A01_ITEM013
422,A01,PG01,P01,ITEM013,201504,2015,4,17,Y,2,N,0.000000,1,0,130,A01_ITEM013
423,A01,PG01,P01,ITEM013,201505,2015,5,7,N,4,N,0.000000,0,0,130,A01_ITEM013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31458,A01,PG05,P16,ITEM465,201701,2017,1,1090,Y,1,Y,0.308385,1,1,110,A01_ITEM465
31459,A01,PG05,P16,ITEM465,201702,2017,2,884,N,4,Y,0.308385,0,1,110,A01_ITEM465
31460,A01,PG05,P16,ITEM465,201703,2017,3,347,Y,2,Y,0.308385,1,1,110,A01_ITEM465
31461,A01,PG05,P16,ITEM465,201712,2017,12,41,N,4,N,0.000000,0,0,110,A01_ITEM465


In [24]:
mlData.IDX_KEY.drop_duplicates() # 머신러닝 돌릴 수 있는 데이터가 251개

419      A01_ITEM013
549      A01_ITEM014
688      A01_ITEM015
828      A01_ITEM016
1060     A01_ITEM018
            ...     
29741    A01_ITEM420
29854    A01_ITEM421
29967    A01_ITEM422
31084    A01_ITEM458
31353    A01_ITEM465
Name: IDX_KEY, Length: 131, dtype: object

In [25]:
mlGroupData = mlData.groupby( by=groupKey )
mlGroupData

In [26]:
# Group by 머신러닝!!

In [ ]:
### 아래 부분 전부 함수로 묶고 return testDataAll 해 주자!!
# def mlFunc( eachGroup ):

In [45]:
# 그룹 데이터 중에 1개를 추출
# 1개의 그룹에 대해서 머신러닝 로직을 구현
# 해당 로직을 함수로 설정
# 각 그룹별 머신러닝을 구동
# mlGroupData = mlData.groupby( by = groupKey ).apply( 해당 함수 ) => 해당 함수를 각 그룹별로 적용
eachGroup = mlGroupData.get_group( list( mlGroupData.groups )[100] ) # 함수 쓰면 이건 디버깅 코드가 됨

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION,QTY_COUNT,IDX_KEY
22501,A01,PG04,P12,ITEM309,201516,2015,16,18,N,4,N,0.000000,0,0,131,A01_ITEM309
22502,A01,PG04,P12,ITEM309,201517,2015,17,68,N,4,N,0.000000,0,0,131,A01_ITEM309
22503,A01,PG04,P12,ITEM309,201518,2015,18,82,N,4,N,0.000000,0,0,131,A01_ITEM309
22504,A01,PG04,P12,ITEM309,201519,2015,19,99,N,4,N,0.000000,0,0,131,A01_ITEM309
22505,A01,PG04,P12,ITEM309,201520,2015,20,116,Y,1,N,0.000000,1,0,131,A01_ITEM309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22627,A01,PG04,P12,ITEM309,201737,2017,37,210,N,4,Y,0.230947,0,1,131,A01_ITEM309
22628,A01,PG04,P12,ITEM309,201738,2017,38,100,N,4,Y,0.154042,0,1,131,A01_ITEM309
22629,A01,PG04,P12,ITEM309,201739,2017,39,180,N,4,Y,0.230947,0,1,131,A01_ITEM309
22630,A01,PG04,P12,ITEM309,201740,2017,40,200,N,4,Y,0.230947,0,1,131,A01_ITEM309


In [46]:
### 2. 특성 선정 / 데이터 분리
### 특성 선정

In [47]:
corrDf = eachGroup.corr(numeric_only=True)
corrDf

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION,QTY_COUNT
YEARWEEK,1.000000,0.984305,-0.203842,-0.436049,0.088780,0.299744,-0.064734,0.301681,NaN
YEAR,0.984305,1.000000,-0.373413,-0.464635,0.137731,0.230227,-0.104622,0.262509,NaN
WEEK,-0.203842,-0.373413,1.000000,0.285434,-0.297383,0.298453,0.240110,0.129544,NaN
QTY,-0.436049,-0.464635,0.285434,1.000000,-0.472368,0.302463,0.418294,0.147960,NaN
HCLUS,0.088780,0.137731,-0.297383,-0.472368,1.000000,-0.499663,-0.976884,-0.086264,NaN
PRO_PERCENT,0.299744,0.230227,0.298453,0.302463,-0.499663,1.000000,0.442608,0.637788,NaN
LE_HOLIDAY,-0.064734,-0.104622,0.240110,0.418294,-0.976884,0.442608,1.000000,0.085882,NaN
LE_PROMOTION,0.301681,0.262509,0.129544,0.147960,-0.086264,0.637788,0.085882,1.000000,NaN
QTY_COUNT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
label = ["QTY"]

In [49]:
stdRatio = [0.5, 0.4, 0.3, 0.2] # 0.5부터 찾고 0.2까지 찾음

In [50]:
import numpy as np
stdRatio = list( np.array( range( 5, 1, -1 ) ) / 10 ) # array의 장점: 그냥 나누기 10 하면 각 요소에 적용됨
stdRatio

[0.5, 0.4, 0.3, 0.2]

In [51]:
features = list( corrDf.loc[ abs(corrDf.QTY) >= stdRatio[0] ].index )
features

['QTY']

In [52]:
for i in range(0, len(stdRatio)):
  features = list( corrDf.loc[ corrDf.QTY >= stdRatio[i] ].index )
  featureLen = len(features)
  if featureLen <= 1:
    continue
  elif featureLen >= 2:
    break
  else:
    print("exceiption")
features

['QTY', 'LE_HOLIDAY']

In [53]:
simulatedRatio = stdRatio[i] # 어디에서 멈췄는지
simulatedRatio

0.4

In [57]:
features = list( set(features) - set(label) )
features

['LE_HOLIDAY']

아래처럼 할 수도 있음

In [55]:
# 1. set으로 문제지 한 번에 빼기

# corrDf.loc[ corrDf.QTY > stdRatio[0] ] # 0.5 이상이 베스트!
list( set( corrDf.loc[ corrDf.QTY >= stdRatio[1] ].index ) - set(label) )

['LE_HOLIDAY']

In [56]:
# 2. list로 문제지 두 번에 나눠서 빼기

# 0.5에서 시작! 만약에 feature를 한 개라도 찾으면 stop 없으면 list에서 다음 기준으로 찾는다 또 한 개라도 찾으면 stop
for i in range(0, len(stdRatio)):
  features = list( corrDf.loc[ corrDf.QTY >= stdRatio[i] ].index )
  features = list( set( features ) - set( label) )
  if len(features) >= 1:
    break;
  else:
    continue;
features

['LE_HOLIDAY']

In [ ]:
### 데이터 분리

In [42]:
from sklearn.model_selection import train_test_split

In [59]:
trainData, testData = train_test_split( eachGroup, test_size=0.2, random_state=1 )

In [77]:
trainingDataFeatures = trainData.loc[ :, features ]
trainingDataLabel = trainData.loc[ :, label]
testDataFeatures = testData.loc[ :, features ]
testDataLabel = testData.loc[ :, label]
testDataAll = testData.loc[ :, ]
print(trainingDataFeatures.shape)
print(trainingDataLabel.shape)
print(testDataFeatures.shape)
print(testDataLabel.shape)
print(testDataAll.shape)
testDataAll

(104, 1)
(104, 1)
(27, 1)
(27, 1)
(27, 16)


,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION,QTY_COUNT,IDX_KEY
22543,A01,PG04,P12,ITEM309,201605,2016,5,365,N,4,Y,0.230947,0,1,131,A01_ITEM309
22581,A01,PG04,P12,ITEM309,201643,2016,43,51,N,4,Y,0.154042,0,1,131,A01_ITEM309
22629,A01,PG04,P12,ITEM309,201739,2017,39,180,N,4,Y,0.230947,0,1,131,A01_ITEM309
22506,A01,PG04,P12,ITEM309,201521,2015,21,159,Y,1,Y,0.154042,1,1,131,A01_ITEM309
22545,A01,PG04,P12,ITEM309,201607,2016,7,372,Y,1,Y,0.230947,1,1,131,A01_ITEM309
22576,A01,PG04,P12,ITEM309,201638,2016,38,55,N,4,Y,0.154042,0,1,131,A01_ITEM309
22603,A01,PG04,P12,ITEM309,201713,2017,13,162,N,4,Y,0.192148,0,1,131,A01_ITEM309
22611,A01,PG04,P12,ITEM309,201721,2017,21,273,Y,1,Y,0.230947,1,1,131,A01_ITEM309
22598,A01,PG04,P12,ITEM309,201708,2017,8,196,Y,1,Y,0.230947,1,1,131,A01_ITEM309
22518,A01,PG04,P12,ITEM309,201533,2015,33,296,N,4,Y,0.154042,0,1,131,A01_ITEM309


### 3. 모델 적용

In [64]:
from sklearn import tree

In [65]:
dtModel = tree.DecisionTreeRegressor(random_state=10)

In [67]:
fittedModel = dtModel.fit(X=trainingDataFeatures, y=trainingDataLabel)

### 4. 예측

In [78]:
predictValue = fittedModel.predict(testDataFeatures)
testDataLabel["DT_PREDICT"] = predictValue
testDataLabel
testDataAll["DT_PREDICT"] = predictValue
testDataAll

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION,QTY_COUNT,IDX_KEY,DT_PREDICT
22543,A01,PG04,P12,ITEM309,201605,2016,5,365,N,4,Y,0.230947,0,1,131,A01_ITEM309,217.96875
22581,A01,PG04,P12,ITEM309,201643,2016,43,51,N,4,Y,0.154042,0,1,131,A01_ITEM309,217.96875
22629,A01,PG04,P12,ITEM309,201739,2017,39,180,N,4,Y,0.230947,0,1,131,A01_ITEM309,217.96875
22506,A01,PG04,P12,ITEM309,201521,2015,21,159,Y,1,Y,0.154042,1,1,131,A01_ITEM309,594.42500
22545,A01,PG04,P12,ITEM309,201607,2016,7,372,Y,1,Y,0.230947,1,1,131,A01_ITEM309,594.42500
22576,A01,PG04,P12,ITEM309,201638,2016,38,55,N,4,Y,0.154042,0,1,131,A01_ITEM309,217.96875
22603,A01,PG04,P12,ITEM309,201713,2017,13,162,N,4,Y,0.192148,0,1,131,A01_ITEM309,217.96875
22611,A01,PG04,P12,ITEM309,201721,2017,21,273,Y,1,Y,0.230947,1,1,131,A01_ITEM309,594.42500
22598,A01,PG04,P12,ITEM309,201708,2017,8,196,Y,1,Y,0.230947,1,1,131,A01_ITEM309,594.42500
22518,A01,PG04,P12,ITEM309,201533,2015,33,296,N,4,Y,0.154042,0,1,131,A01_ITEM309,217.96875


In [80]:
testDataAll["STDRATIO"] = simulatedRatio
testDataAll

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,LE_HOLIDAY,LE_PROMOTION,QTY_COUNT,IDX_KEY,DT_PREDICT,STDRATIO
22543,A01,PG04,P12,ITEM309,201605,2016,5,365,N,4,Y,0.230947,0,1,131,A01_ITEM309,217.96875,0.4
22581,A01,PG04,P12,ITEM309,201643,2016,43,51,N,4,Y,0.154042,0,1,131,A01_ITEM309,217.96875,0.4
22629,A01,PG04,P12,ITEM309,201739,2017,39,180,N,4,Y,0.230947,0,1,131,A01_ITEM309,217.96875,0.4
22506,A01,PG04,P12,ITEM309,201521,2015,21,159,Y,1,Y,0.154042,1,1,131,A01_ITEM309,594.42500,0.4
22545,A01,PG04,P12,ITEM309,201607,2016,7,372,Y,1,Y,0.230947,1,1,131,A01_ITEM309,594.42500,0.4
22576,A01,PG04,P12,ITEM309,201638,2016,38,55,N,4,Y,0.154042,0,1,131,A01_ITEM309,217.96875,0.4
22603,A01,PG04,P12,ITEM309,201713,2017,13,162,N,4,Y,0.192148,0,1,131,A01_ITEM309,217.96875,0.4
22611,A01,PG04,P12,ITEM309,201721,2017,21,273,Y,1,Y,0.230947,1,1,131,A01_ITEM309,594.42500,0.4
22598,A01,PG04,P12,ITEM309,201708,2017,8,196,Y,1,Y,0.230947,1,1,131,A01_ITEM309,594.42500,0.4
22518,A01,PG04,P12,ITEM309,201533,2015,33,296,N,4,Y,0.154042,0,1,131,A01_ITEM309,217.96875,0.4


In [ ]:
return testDataAll